# **Cloning from Repository, Settings**

In [ ]:
from google.colab import gdrive
drive.mount('/content/gdrive/')

cd /content/gdrive/My drive/data

In [1]:
import os
import io
import sys
import argparse
import json
import numpy as np
import glob
import PIL.Image
import requests

json_file = os.path.abspath("../content/mask.json")
ROOT_DIR = os.path.abspath("../content/drive/MyDrive/availabler_mask/")

required_images = ["023","035","037","046","049","058","060","062","063","065","066","069","072","075","077","088","090","091","092","094","095","096","097","102","108","123","124","128","130","131","133","134","136","144","146","151","152","153","161","162","163","171","188","191","193","194","195","198","206","208","218","220"]
with open(json_file, "r") as f:
  data = json.load(f)

for single_image in data :
  image_name = single_image["External ID"]
  image_name = ''.join(image_name.split())[:-4]
  if image_name[-3:] in required_images:
    DIR=os.path.join(ROOT_DIR, image_name)
    if os.path.isdir(DIR)==False:
      os.mkdir(DIR)
    
    if not single_image["Label"] == {}:
      for i, mask in enumerate(single_image["Label"]["objects"]):
        mask_name=mask["instanceURI"]
        response = requests.get(mask["instanceURI"])
        image_bytes = io.BytesIO(response.content)
        img = PIL.Image.open(image_bytes)

        img = img.convert("RGBA")
        datas = img.getdata()
        newData = []
        for item in datas:
          if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((0, 0,0, 255))
          else:
            newData.append((0, 0, 0, 0))
        img.putdata(newData)
        sys.path.append(DIR)
        img.save(os.path.join(DIR,f"result{i}.png"), "PNG")
    